# Topic Modeling

The goal of this project is to **assign over 400 000 quora questions to different categories**, or topics.

For that, we'll be using three different methods:
* **Latent Dirichlet Allocation (LDA)**
* **Latent Semantic Analysis (LSA)**
* **Non-Negative Matrix Factorization (NMF)**

#### 1. Perform initial imports

In [1]:
import pandas as pd

#### 2. Load data

In [2]:
quora = pd.read_csv("data/quora_questions.csv")

#### 3. Check the dataframe

In [3]:
quora.head()

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


#### 4. Check missing values

In [4]:
quora.isnull().sum()

Question    0
dtype: int64

There are no missing questions.

#### 5. Check empty strings

In [5]:
# using the isspace() method

empty_strings = []

for i, q in quora.itertuples():
    if q.isspace():
        empty_strings.append(i)

In [6]:
print(empty_strings)
print(len(empty_strings))

[]
0


There are no questions that correspond to empty strings.

In [7]:
# check length

len(quora)

404289

We have 404 289 quora questions. Our dataset is cleaned and we can now perform topic modeling with LDA, LSA and NMF.

## LDA

In Latent Dirichlet Allocation, topics are represented as a distribution of words, i. e., the probability that each of a given set of terms will occur. Documents are in turn represented as a mixture (linear combination) of these topics. The probability for each of these topics within a document, as well as the probability of a word being assigned to a topic, is assumed to start with a Dirichlet probability distribution.

#### 7. Create a vectorized document-term matrix with `CountVectorizer`

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [10]:
dtm = cv.fit_transform(quora['Question'])

In [11]:
dtm

<404289x38669 sparse matrix of type '<class 'numpy.int64'>'
	with 2002912 stored elements in Compressed Sparse Row format>

We now have a sparse matrix for the 404 289 questions for a total of 38 669 different words. Let's try to  group these questions into 20 different topics with the LDA method.

#### 8. Create an instance of LatentDirichletAllocation with 20 expected components and fit it

In [12]:
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
LDA = LatentDirichletAllocation(n_components=20,random_state=42)

In [14]:
%%time

LDA.fit(dtm)

Wall time: 8min 5s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

#### 9. Check how we can have access to words and topics 

In [15]:
# words

print(len(cv.get_feature_names()))

38669


In [16]:
import random

for i in range(10):
    random_word_id = random.randint(0,38668)
    print(cv.get_feature_names()[random_word_id])

spoil
smayan
close
stops
candidature
journeys
annie
stories
rabid
crunchbase


Like we have seen before, we have a total of 38 669 different words. We've print out 10 random words of those 38 669 words.

In [17]:
# topics

len(LDA.components_)

20

In [18]:
# words

len(LDA.components_[0])

38669

As expected, we have 20 different topics. And for each topic, we have a certain combination of our total of 38 669 words.

In [19]:
# top 10 words for topic #0 in descending order

top10_word_indices = LDA.components_[0].argsort()[-10:][::-1] #[::-1] to reverse order

for index in top10_word_indices:
    print(cv.get_feature_names()[index])

best
service
history
career
social
india
company
good
google
media


These are the top 10 words for topic #0.

#### 10. Print out the top 15 words for each of the 20 topics

In [20]:
for index,topic in enumerate(LDA.components_):
    print(f'TOP 15 WORDS FOR TOPIC #{index+1}') #index+1 to start with Topic #1 instead of #0
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]][::-1])
    print('\n')

TOP 15 WORDS FOR TOPIC #1
['best', 'service', 'history', 'career', 'social', 'india', 'company', 'good', 'google', 'media', 'services', 'open', 'code', 'development', 'sydney']


TOP 15 WORDS FOR TOPIC #2
['500', 'notes', '1000', 'indian', 'black', 'english', 'rs', 'money', 'word', 'india', 'rupee', 'government', 'making', 'process', 'economy']


TOP 15 WORDS FOR TOPIC #3
['does', 'average', 'cost', 'india', 'good', 'purpose', 'man', 'compare', 'state', 'home', 'legal', 'center', 'alcohol', 'ones', 'current']


TOP 15 WORDS FOR TOPIC #4
['new', 'iphone', 'does', 'tv', 'exist', 'big', 'worth', 'interesting', 'looking', 'series', 'mind', 'apple', 'facts', 'year', 'answers']


TOP 15 WORDS FOR TOPIC #5
['job', 'car', 'india', 'differences', 'college', 'jobs', 'apply', 'mba', 'canada', 'visa', 'student', 'students', 'usa', 'overcome', 'australia']


TOP 15 WORDS FOR TOPIC #6
['world', 'long', 'does', 'war', 'india', 'math', 'like', 'china', 'pakistan', 'countries', 'culture', 'relationship

#### 11. Visualize topics and most relevant terms per topic

In [22]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [23]:
%%time

pyLDAvis.sklearn.prepare(LDA, dtm, cv, sort_topics=False)
# sort_topics = False to keep the original topic order

Wall time: 18min 39s


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.101456  0.162730       1        1  4.802104
1      0.056435  0.004201       2        1  4.763971
2      0.139238 -0.031562       3        1  4.356176
3      0.061751  0.053937       4        1  4.317269
4     -0.003989  0.079946       5        1  5.482495
5      0.182879 -0.011909       6        1  5.017840
6     -0.176029 -0.190660       7        1  5.094417
7     -0.219222 -0.050482       8        1  5.499448
8     -0.063818  0.237554       9        1  5.251726
9      0.107664 -0.087074      10        1  4.263859
10    -0.066232  0.130190      11        1  4.741658
11     0.015466  0.123594      12        1  5.114491
12    -0.017193  0.021418      13        1  5.291642
13     0.033527  0.056563      14        1  4.545807
14     0.179254 -0.009268      15        1  4.956969
15    -0.110974  0.033650      16        1  4.632987
16    -0.113493 -0.171297      17        1  4.753560
17     0.129832 -0.087116      18        1  4.751397
18    -0.188542 -0.125629      19        1  5.520167
19     0.154902 -0.138788      20        1  6.842018, topic_info=      Category          Freq    Term         Total  loglift  logprob
11005  Default  27886.000000    does  27886.000000  30.0000  30.0000
4632   Default  33067.000000    best  33067.000000  29.0000  29.0000
25777  Default  12338.000000  people  12338.000000  28.0000  28.0000
10447  Default   9165.000000     did   9165.000000  27.0000  27.0000
21116  Default   8693.000000    make   8693.000000  26.0000  26.0000
...        ...           ...     ...           ...      ...      ...
15060  Topic20   1507.933622    good  11755.178156   0.6285  -4.5371
34869  Topic20   1186.306488    time   7099.106158   0.8930  -4.7770
14185  Topic20    738.475243  friend   1541.144560   1.9464  -5.2510
15914  Topic20    680.806255  having   1488.632903   1.8998  -5.3323
11065  Topic20    714.229200     don   3649.823720   1.0509  -5.2844

[1186 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
1          4  0.059751      000
1         16  0.938584      000
34         1  0.023685       10
34         4  0.019130       10
34         6  0.025507       10
...      ...       ...      ...
38340     20  0.993607   yellow
38399      4  0.998505     york
38422     10  0.976728  youtube
38422     16  0.023012  youtube
38558      4  0.993598     zone

[2824 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

With $\lambda=1$, the terms are ranked by their probabilities within each topic (as we had in our top 15 words for each of the 20 topics). With $\lambda=0$, the terms are ranked only by their lift, the ratio between a term's probability within a topic and its margin probability across the corpus. For other values of $\lambda$, the terms are ranked by their relevance, a combination of those two values.

As we can see in our intertopic distance map, there is **some overlaping between different topics**. Let's try to avoid this overlaping by **reducing the total number of topics to 15**.

In [27]:
LDA = LatentDirichletAllocation(n_components=15,random_state=42)

In [28]:
%%time

LDA.fit(dtm)

Wall time: 9min 30s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=15, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [29]:
%%time

pyLDAvis.sklearn.prepare(LDA, dtm, cv, sort_topics=False)

Wall time: 19min 44s


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.102309 -0.109533       1        1  6.090068
1      0.080048  0.054730       2        1  5.961436
2      0.151624  0.072886       3        1  6.069565
3      0.058468 -0.183375       4        1  5.895824
4      0.000703 -0.026767       5        1  7.090827
5      0.219390  0.024418       6        1  8.417809
6     -0.133039  0.229527       7        1  7.032200
7     -0.266418  0.136075       8        1  8.671695
8     -0.138725 -0.150962       9        1  6.574202
9      0.083366  0.141939      10        1  5.648120
10    -0.092651 -0.183004      11        1  5.775134
11    -0.029628 -0.108231      12        1  6.703528
12     0.016564  0.090505      13        1  7.056417
13    -0.030070  0.051972      14        1  6.012235
14     0.182675 -0.040179      15        1  7.000940, topic_info=      Category          Freq     Term         Total  loglift  logprob
4632   Default  32260.000000     best  32260.000000  30.0000  30.0000
11005  Default  27918.000000     does  27918.000000  29.0000  29.0000
25777  Default  12575.000000   people  12575.000000  28.0000  28.0000
20333  Default  12084.000000     like  12084.000000  27.0000  27.0000
10447  Default   9116.000000      did   9116.000000  26.0000  26.0000
...        ...           ...      ...           ...      ...      ...
4525   Topic15    706.050826  believe   1351.747932   2.0097  -5.3189
17511  Topic15   1118.820112   indian   5343.140569   1.0956  -4.8585
11005  Topic15   1720.071011     does  27918.238452  -0.1278  -4.4284
15769  Topic15    818.772826   happen   2417.078687   1.5766  -5.1707
17507  Topic15   1145.042939    india  13978.499831   0.1570  -4.8354

[895 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
1          2  0.006141      000
1          4  0.024562      000
1          8  0.132637      000
1         14  0.836348      000
34         1  0.033551       10
...      ...       ...      ...
38422      8  0.077261  youtube
38422      9  0.276853  youtube
38422     10  0.442965  youtube
38422     14  0.202811  youtube
38558      4  0.996585     zone

[1990 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

#### 12. Create a dataframe with each question and the correspoding topic

In [30]:
topic_results = LDA.transform(dtm)

In [31]:
topic_results.shape

(404289, 15)

In [32]:
# for question #2

topic_results[2]

array([0.00952381, 0.00952381, 0.00952381, 0.00952381, 0.00952381,
       0.00952381, 0.00952381, 0.00952381, 0.72894152, 0.00952388,
       0.00952381, 0.00952381, 0.00952381, 0.00952381, 0.14724888])

In [33]:
topic_results[2].round(2)

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.73, 0.01, 0.01,
       0.01, 0.01, 0.01, 0.15])

In [34]:
topic_results[2].argmax()

8

According to this, question #2 belongs to topic #8 (if we start counting from zero).

In [35]:
# for all questions

quora_lda = quora.copy()
quora_lda['Topic'] = topic_results.argmax(axis=1)+1 # +1 so that we have topics 1 to 15

In [38]:
quora_lda.head()

,Question,Topic
0,What is the step by step guide to invest in sh...,8
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,5
2,How can I increase the speed of my internet co...,9
3,Why am I mentally very lonely? How can I solve...,11
4,"Which one dissolve in water quikly sugar, salt...",3


We've managed to assign each question to one of the 15 topics. Let's do the same thing with the LSA method.

## LSA

In Latent Semantic Analysis, we can break down our TF-IDF document-term matrix into **three simpler matrices** using singular value decomposition (SVD). We can then truncate those matrices (remove some rows and columns), which reduces the number of dimensions we have to deal with in our vector space model - this is called **truncated singular value decomposition**.

If we multiply these new truncated matrices, we don't get the exact same TF-IDF matrix, but a new representation of the documents that contains the esence or "latent semantics" of those documents.

#### 13. Create a vectorized document-term matrix with `TfidfVectorizer`

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [42]:
dtm = tfidf.fit_transform(quora['Question'])

In [43]:
dtm

<404289x38669 sparse matrix of type '<class 'numpy.float64'>'
	with 2002912 stored elements in Compressed Sparse Row format>

We have now used `TfidfVectorizer` instead of `CountVectorizer` to create our document-term matrix.

#### 14. Create an instance of TruncatedSVD with 15 expected components and fit it

In [44]:
from sklearn.decomposition import TruncatedSVD

In [45]:
lsa_model = TruncatedSVD(n_components=15, random_state=42)

In [46]:
%%time

lsa_model.fit(dtm)

Wall time: 3.59 s


TruncatedSVD(algorithm='randomized', n_components=15, n_iter=5, random_state=42,
             tol=0.0)

#### 15. Print out the top 15 words for each of the 15 topics

In [51]:
for index,topic in enumerate(lsa_model.components_):
    print(f'TOP 15 WORDS FOR TOPIC #{index+1}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]][::-1])
    print('\n')

TOP 15 WORDS FOR TOPIC #1
['best', 'way', 'learn', 'india', 'money', 'make', 'does', 'online', 'quora', 'life', 'books', 'ways', 'book', 'programming', 'language']


TOP 15 WORDS FOR TOPIC #2
['does', 'quora', 'money', 'make', 'people', 'like', 'mean', 'questions', 'feel', 'life', 'work', 'online', 'question', 'earn', 'ask']


TOP 15 WORDS FOR TOPIC #3
['quora', 'questions', 'people', 'question', 'ask', 'money', 'answers', 'answer', 'google', 'make', 'asked', 'online', 'easily', 'earn', 'delete']


TOP 15 WORDS FOR TOPIC #4
['money', 'make', 'online', 'earn', '500', 'way', '1000', 'notes', 'ways', 'black', 'youtube', 'india', 'rupee', 'easy', 'rs']


TOP 15 WORDS FOR TOPIC #5
['life', 'people', 'india', 'trump', 'know', 'donald', 'think', 'good', 'like', 'did', 'purpose', 'things', 'love', 'thing', 'important']


TOP 15 WORDS FOR TOPIC #6
['india', 'trump', 'donald', 'people', 'president', 'clinton', 'hillary', 'think', 'win', '500', 'notes', '1000', 'did', 'election', 'pakistan']


TO

#### 16. Create a dataframe with each question and the correspoding topic

In [53]:
topic_results = lsa_model.transform(dtm)

In [54]:
quora_lsa = quora.copy()

In [55]:
quora_lsa['Topic'] = topic_results.argmax(axis=1)+1

In [56]:
quora_lsa.head()

,Question,Topic
0,What is the step by step guide to invest in sh...,6
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,14
2,How can I increase the speed of my internet co...,2
3,Why am I mentally very lonely? How can I solve...,2
4,"Which one dissolve in water quikly sugar, salt...",2


It's now time do the same thing with the NMF method.

## NMF

With Non-Negative Matrix Factorization, our TF-IDF document-term matrix is now decomposed into **two factors** whose product approximates the original, in a way that every value in both factors is either positive or zero. These two matrices represent topics positively related to terms and documents of the corpus.

#### 17. Create an instance of NMF with 15 expected components and fit it

In [57]:
from sklearn.decomposition import NMF

In [58]:
nmf_model = NMF(n_components=15, random_state=42)

In [59]:
%%time

nmf_model.fit(dtm)

Wall time: 37 s


NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=15, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

#### 18. Print out the top 15 words for each of the 15 topics

In [60]:
for index,topic in enumerate(nmf_model.components_):
    print(f'TOP 15 WORDS FOR TOPIC #{index+1}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]][::-1])
    print('\n')

TOP 15 WORDS FOR TOPIC #1
['best', 'way', 'movies', 'book', 'books', '2016', 'movie', 'laptop', 'buy', 'ways', 'time', 'phone', 'places', 'visit', 'place']


TOP 15 WORDS FOR TOPIC #2
['does', 'mean', 'work', 'feel', 'long', 'cost', 'compare', 'really', 'time', 'exist', 'sex', 'use', 'looking', 'differ', 'recruit']


TOP 15 WORDS FOR TOPIC #3
['quora', 'questions', 'question', 'ask', 'answer', 'answers', 'google', 'asked', 'delete', 'improvement', 'easily', 'post', 'needing', 'answered', 'add']


TOP 15 WORDS FOR TOPIC #4
['money', 'make', 'online', 'earn', 'way', 'ways', 'youtube', 'easy', 'home', 'easiest', 'free', 'internet', 'black', 'friends', 'facebook']


TOP 15 WORDS FOR TOPIC #5
['life', 'purpose', 'meaning', 'thing', 'important', 'real', 'moment', 'change', 'want', 'live', 'day', 'changed', 'death', 'did', 'earth']


TOP 15 WORDS FOR TOPIC #6
['india', 'pakistan', 'war', 'world', 'start', 'spotify', 'country', 'job', 'business', 'available', 'olympics', 'china', 'engineering'

#### 19. Create a dataframe with each question and the correspoding topic

In [61]:
topic_results = nmf_model.transform(dtm)

In [62]:
quora_nmf = quora.copy()

In [63]:
quora_nmf['Topic'] = topic_results.argmax(axis=1)+1

In [64]:
quora_nmf.head()

,Question,Topic
0,What is the step by step guide to invest in sh...,6
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,9
2,How can I increase the speed of my internet co...,4
3,Why am I mentally very lonely? How can I solve...,12
4,"Which one dissolve in water quikly sugar, salt...",15


We've managed to assign each question to its corresponding topic using **three different methods**. So, which is the best model? As is almost always the case, it depends. We have to consider the corpus we are dealing with and the goal we're trying to achieve.

Generally speaking, **LDA is better when you want easier to explain topic vectors**, but **LSA and NMF are usually much faster to compute**.